In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data preprocessing

In [2]:
from sklearn.impute import SimpleImputer
data = pd.read_csv('data/tested.csv')
missing_columns = data.columns[data.isnull().any()]


Index(['Age', 'Fare', 'Cabin'], dtype='object')